In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
import category_encoders as ce
from pathlib import Path

In [5]:
project_root_dir = Path('/Users/angelo/Programming/Python/scikit-learn-exercises/adult_income/')
adult_data_file = project_root_dir / 'adult.data'

In [7]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status',
                'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country','income']
adults_data = pd.read_csv(adult_data_file, names=column_names)

In [18]:
adults_data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [21]:
adults_data.select_dtypes(include='object').nunique()

workclass          9
education         16
marital-status     7
occupation        15
relationship       6
race               5
gender             2
native-country    42
income             2
dtype: int64

In [22]:
adults_data['education'].unique()

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [8]:
adults_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1   workclass        32561 non-null  object
 2   fnlwgt           32561 non-null  int64 
 3   education        32561 non-null  object
 4   educational-num  32561 non-null  int64 
 5   marital-status   32561 non-null  object
 6   occupation       32561 non-null  object
 7   relationship     32561 non-null  object
 8   race             32561 non-null  object
 9   gender           32561 non-null  object
 10  capital-gain     32561 non-null  int64 
 11  capital-loss     32561 non-null  int64 
 12  hours-per-week   32561 non-null  int64 
 13  native-country   32561 non-null  object
 14  income           32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [10]:
numeric_features = adults_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = adults_data.select_dtypes(include=['object']).drop(['income'], axis=1).columns

In [14]:
X = adults_data.drop('income', axis=1)
y = adults_data['income']

In [15]:
le = preprocessing.LabelEncoder()
label_encoder = le.fit(y)
y = label_encoder.transform(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
encoder_list = [ce.backward_difference.BackwardDifferenceEncoder, 
                ce.basen.BaseNEncoder,
                ce.binary.BinaryEncoder,
                ce.cat_boost.CatBoostEncoder,
                ce.hashing.HashingEncoder,
                ce.helmert.HelmertEncoder,
                ce.james_stein.JamesSteinEncoder,
                ce.one_hot.OneHotEncoder,
                ce.leave_one_out.LeaveOneOutEncoder,
                ce.m_estimate.MEstimateEncoder,
                ce.ordinal.OrdinalEncoder,
                ce.polynomial.PolynomialEncoder,
                ce.sum_coding.SumEncoder,
                ce.target_encoder.TargetEncoder,
                ce.woe.WOEEncoder]

In [23]:
for encoder in encoder_list:
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                              ('woe', encoder())])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                   ('cat', categorical_transformer, categorical_features)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier(n_estimators=500))])
    
    model = pipe.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(encoder)
    print(f1_score(y_test, y_pred, average='macro'))

<class 'category_encoders.backward_difference.BackwardDifferenceEncoder'>
0.7931218847494601
<class 'category_encoders.basen.BaseNEncoder'>
0.7973411209509889
<class 'category_encoders.binary.BinaryEncoder'>
0.7974127996601739
<class 'category_encoders.cat_boost.CatBoostEncoder'>
0.8063826816672213
<class 'category_encoders.hashing.HashingEncoder'>
0.7695167500524795
<class 'category_encoders.helmert.HelmertEncoder'>
0.7922320864042967
<class 'category_encoders.james_stein.JamesSteinEncoder'>
0.801602515065982
<class 'category_encoders.one_hot.OneHotEncoder'>
0.8004378260766312
<class 'category_encoders.leave_one_out.LeaveOneOutEncoder'>
0.4319727891156463
<class 'category_encoders.m_estimate.MEstimateEncoder'>
0.8043573261467001
<class 'category_encoders.ordinal.OrdinalEncoder'>
0.8023731321274491
<class 'category_encoders.polynomial.PolynomialEncoder'>
0.7915412233813167
<class 'category_encoders.sum_coding.SumEncoder'>
0.8004874471820537
<class 'category_encoders.target_encoder.Targ